In [1]:
import pprint as pp
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import simplejson as json
import time
import random
import os
import glob
import re
import unidecode
import datetime as dt
from collections import Counter
import sys
import pickle
import codecs
import urllib

In [2]:
df = pd.DataFrame(columns=['rider_name','season','race','racetype', 'racedays'])

In [3]:
tour_url = "tour-de-france"
giro_url = "giro-d-italia"
vuelta_url = "vuelta-a-espana"
strade_url = "strade-bianchi"
paris_nice_url = "paris-nice"
tirreno_url = "tirreno-adriatico"
dauphine_url = "dauphine"
sanremo_url = "milano-sanremo"
gent_wevelgem_url = "gent-wevelgem"
ronde_url = "ronde-van-vlaanderen"
roubaix_url = "paris-roubaix"
amstel_url = "amstel-gold-race"
california_url = "tour-of-california"
liege_url = "liege-bastogne-liege"
suisse_url = "tour-de-suisse"
lombardia_url = "il-lombardia"
romandie_url = "tour-de-romandie"
race_url_list = ["tour-de-france",
                 "giro-d-italia",
                 "vuelta-a-espana",
                 "strade-bianchi",
                 "paris-nice",
                 "tirreno-adriatico",
                 "dauphine",
                 "milano-sanremo",
                 "gent-wevelgem",
                 "ronde-van-vlaanderen",
                 "paris-roubaix",
                 "amstel-gold-race",
                 "tour-of-california",
                 "liege-bastogne-liege",
                 "tour-de-suisse",
                 "il-lombardia",
                 "tour-de-romandie"]

In [4]:
user_agent = {'User-agent': 'Mozilla/5.0'}

def get_rider_urls(race_url,start_year,years_back):
    base_url = 'https://www.procyclingstats.com/race/' + str(race_url) + '/'
    print(str(race_url).upper(),'starting...')
    
    for year in range(int(start_year),int(start_year-(years_back+1)),-1):
        url = base_url + str(year) + '/startlist'
        response = requests.get(url, headers=user_agent)

        if response.status_code == 200:
            page = response.text
            soup = BeautifulSoup(page, 'lxml')
            startlist = soup.find_all('a', class_='blue')
            urls = [x['href'] for x in startlist]
            urls = ['https://www.procyclingstats.com/' + u for u in urls]
            file = str(race_url) + '_' + str(year) + '.json'
            with open(file, 'w') as f:
                json.dump(urls, f)
            timer = 2 + 2 * random.random()
            print(year,'done, sleeping for',np.round(timer,2),'sec')
            time.sleep(timer)
        else:
            print('unsuccessful request!')
            print('status code:',response.status_code)
            break
    print('finished!\n')

In [5]:
for race_url in race_url_list:
    get_rider_urls(race_url,2019,5)

TOUR-DE-FRANCE starting...
2019 done, sleeping for 3.73 sec
2018 done, sleeping for 3.16 sec
2017 done, sleeping for 3.08 sec
2016 done, sleeping for 2.75 sec
2015 done, sleeping for 3.68 sec
2014 done, sleeping for 2.43 sec
finished!

GIRO-D-ITALIA starting...
2019 done, sleeping for 3.22 sec
2018 done, sleeping for 2.32 sec
2017 done, sleeping for 3.57 sec
2016 done, sleeping for 2.15 sec
2015 done, sleeping for 2.38 sec
2014 done, sleeping for 2.84 sec
finished!

VUELTA-A-ESPANA starting...
2019 done, sleeping for 2.92 sec
2018 done, sleeping for 2.39 sec
2017 done, sleeping for 3.39 sec
2016 done, sleeping for 3.76 sec
2015 done, sleeping for 2.16 sec
2014 done, sleeping for 3.04 sec
finished!

STRADE-BIANCHI starting...
2019 done, sleeping for 2.99 sec
2018 done, sleeping for 3.86 sec
2017 done, sleeping for 2.17 sec
2016 done, sleeping for 3.25 sec
2015 done, sleeping for 3.79 sec
2014 done, sleeping for 2.02 sec
finished!

PARIS-NICE starting...
2019 done, sleeping for 3.8 sec
2

KeyboardInterrupt: 

In [6]:
unique_riders = []
filenames = []

for race_url in race_url_list:
    for i in range (2014,2020):
        filename = race_url +'_' +  str(i) + '.json'
        filenames.append(filename)

for filename in filenames:
    with open(filename) as f:
        riders = json.load(f)
        unique_riders.extend(riders)

print('number of unique rider urls:',len(set(unique_riders)))
unique_riders = list(set(unique_riders))

number of unique rider urls: 1444


In [10]:
print(dt.datetime.now(),'Downloading web pages...')
start = 1
end = 30
all_filenames = []

for rdx, first_url in enumerate(unique_riders[start:end]):
    # save first season and get list of remaining ones
    response = requests.get(first_url, headers=user_agent)
    page = response.text
    soup = BeautifulSoup(page, 'lxml')
    seasons = soup.find_all('a', class_='seasonResults')
    u = 0
    for i in range (len(seasons)):
        seasons[i] = str(seasons[i])[38:42]
    seasons = list(dict.fromkeys(seasons))
    save_name = first_url.split('/')[4] + '_' + seasons[u] + '.html'
    u+=1
    all_filenames.append(save_name)
    rdx_str = 'R[' + str(rdx+1) + ']'
    sdx_str = 'S[1]'
    
    with open(os.path.join('riders/',save_name), 'w', encoding = 'utf-8') as file:
        file.write(str(soup))
    print(dt.datetime.now(),rdx_str,sdx_str,'Saved:',save_name,end='\r')
    
    random_wait = 2 + 2 * random.random()
    time.sleep(random_wait)
    
    
    # save the remaining seasons
    if len(seasons) > 1:
        remaining_seasons = seasons[1:len(seasons)]
        for sdx, year in enumerate(remaining_seasons):
            next_url = first_url + '/' + year
            response = requests.get(next_url, headers=user_agent)
            page = response.text
            soup = BeautifulSoup(page, 'lxml')
            save_name = first_url.split('/')[4] + '_' + seasons[u]+ '.html'
            u+=1
            sdx_str = 'S[' + str(sdx+2) + ']'
            
            with open(os.path.join('riders/',save_name), 'w', encoding = 'utf-8') as file:
                file.write(str(soup))
            print(dt.datetime.now(),rdx_str,sdx_str,'Saved:',save_name,end='\r')
            all_filenames.append(save_name)
            
            random_wait = 2 + 2 * random.random()
            time.sleep(random_wait)
    else:
        continue2

print(dt.datetime.now(),'Done')

2020-12-05 21:21:17.088859 Downloading web pages...
2020-12-05 21:42:09.523413 Done] S[18] Saved: mickael-delage_2002.htmlllll


In [14]:
for filename in all_filenames:
    soup = BeautifulSoup(open(os.getcwd()+'/riders/'+filename, encoding="utf8"), 'html.parser')
    
    rider_name = str(soup.title)
    rider_name = rider_name[7:-8]
    rider_name = rider_name.lower()
    rider_name = rider_name.replace(" ", "_")
    
    season = filename[-9:-5]
    print(season)
    
    print(rider_name)
    
    links = soup.find_all('a', href=True)
    links
    urls = [x['href'] for x in links]
    
    results = []
    races = []
    racetype = []
    racedays = []

    for i in range(len(urls)):
        if urls[i][-6:] == "Filter":
            results.append(urls[i-1])


    for i in range(len(results)):
        if results[i][-2:] == 'gc':
            races.append(results[i][5:-8])
            racetype.append('gc')
            if i+1 < len(results):
                racedays.append(results[i+1][-1:])
            else:
                racedays.append(0)
            
        if results[i][-2:] == 'lt':
            races.append(results[i][5:-12])
            racetype.append('lt')
            racedays.append(1)
    for i in range (len(races)):
        df.loc[len(df.index)] = [rider_name, season, races[i], racetype[i], racedays[i]] 

2020
stepan_kurianov
2019
stepan_kurianov
2018
stepan_kurianov
2017
stepan_kurianov
2016
stepan_kurianov
2015
stepan_kurianov
2014
stepan_kurianov
2013
stepan_kurianov
2019
ian_boswell
2018
ian_boswell
2017
ian_boswell
2016
ian_boswell
2015
ian_boswell
2014
ian_boswell
2013
ian_boswell
2012
ian_boswell
2011
ian_boswell
2010
ian_boswell
2009
ian_boswell
2008
ian_boswell
2006
ian_boswell
2020
hernán_ricardo_aguirre
2019
hernán_ricardo_aguirre
2018
hernán_ricardo_aguirre
2017
hernán_ricardo_aguirre
2016
hernán_ricardo_aguirre
2015
hernán_ricardo_aguirre
2014
hernán_ricardo_aguirre
2020
fabio_felline
2019
fabio_felline
2018
fabio_felline
2017
fabio_felline
2016
fabio_felline
2015
fabio_felline
2014
fabio_felline
2013
fabio_felline
2012
fabio_felline
2011
fabio_felline
2010
fabio_felline
2009
fabio_felline
2008
fabio_felline
2007
fabio_felline
2016
davide_malacarne
2015
davide_malacarne
2014
davide_malacarne
2013
davide_malacarne
2012
davide_malacarne
2011
davide_malacarne
2010
davide_malac

In [15]:
df

,rider_name,season,race,racetype,racedays
0,stepan_kurianov,2020,paris-tours,lt,1
1,stepan_kurianov,2020,paris-chauny-classique,lt,1
2,stepan_kurianov,2020,okolo-slovenska,gc,4
3,stepan_kurianov,2020,trofeo-matteotti,lt,1
4,stepan_kurianov,2020,milano-sanremo,lt,1
...,...,...,...,...,...
5287,mickaël_delage,2005,omloop-het-nieuwsblad,lt,1
5288,mickaël_delage,2005,classic-haribo,lt,1
5289,mickaël_delage,2005,tour-mediterraneen,gc,5
5290,mickaël_delage,2005,tour-down-under,gc,6


In [16]:
df.to_csv('networktest.csv') 